In [2]:
!pip install billboard.py

In [64]:
import billboard
import re
for curr in range(2019,2023):
    chart = billboard.ChartData('hot-100-songs', year=curr)
    with open("billboard/hot_100_"+str(curr)+".txt",'w') as output:
        output.write("position,artist,song"+"\n")
        for entry in chart.entries:
            type(entry.artist)
            finalartist = entry.artist
            if "&" in entry.artist:
                artist1 = re.split("\s*&",entry.artist)
                finalartist = ' and '.join(artist1)
            if "Featuring" in finalartist:
                artist2 = re.split("Featuring",finalartist)
                finalartist = ' and '.join(artist2)
            #print(entry.title)
            output.write(entry.title+" and "+finalartist+" and "+str(entry.rank)+"\n")
            #print(entry.artist)
        output.close

In [59]:
import os
import re
from io import StringIO
from pathlib import Path
import pandas as pd
path_base = "C:\\Users\\16462\\OneDrive\\Desktop\\assignments\\PA\\Billboard\\billboard\\"

data_map = {}

start_year=1950
end_year=1951
# Get Data as [Year, DF] combo
for i in range (start_year, end_year):
    filename = path_base+str(i)+".csv"
    with open(filename) as file:
        lines = [line.encode('utf-8', errors='ignore').decode('utf-8') for line in file]
        df = pd.read_csv(StringIO('\n'.join(lines)), header=0)
        data_map[i] = df

In [61]:
# get relevant data from Spotify API for the same
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


client_id = '16a3911229cf4fc99361a96d524e6a99'
client_secret = '6038b52110c24d3fac28a5ef1178ad47'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

df_1950 = data_map[1950]


def generate_artists(artist_list):
    # Generate all possible artists with break points
    # feat. / comma / and / with
    return [x.strip() for x in re.split('feat.|and|with|,', artist_list)]

def handle_special_cases(artist, track):
    # Try and remove and search by each artist sequentially
    artist_list = generate_artists(artist_list=artist)
    for curr_artist in artist_list:
        track_id = sp.search(q='artist: '+ curr_artist +' track:' + track, type='track')
        base_data = track_id['tracks']['items']
        if(base_data):
                return base_data[0]['id']
    # Try without artist data as last straw
    track_id = sp.search(q=' track:' + track, type='track')
    base_data =  track_id['tracks']['items']
    if(not base_data):
        return 0
    return base_data[0]['id']

def generate_track_ids(df):
    final_lst = []
    for ind in df.index:
        artist = df['Artist'][ind]
        track =  df['Song Title'][ind]
        track_id = sp.search(q='artist: '+ artist +' track:' + track, type='track')
        base_data = track_id['tracks']['items']
        if(not base_data):
            final_lst.append(handle_special_cases(artist=artist, track=track))
            continue
        final_lst.append(base_data[0]['id'])
    return final_lst


def generate_components_from_track_ids(df):
    track_id_lst = df['spotify_track_ids']
    df_features = pd.DataFrame(columns= ['danceability', 'energy', 'speechiness', 'acousticness', 'liveness'])
    for id in track_id_lst:
        base_features = sp.audio_features(id)[0]
        dict_data = {'danceability' : base_features['danceability'], 'energy': base_features['energy'],'speechiness': base_features['speechiness'],'acousticness': base_features['acousticness'], 'liveness': base_features['liveness']}
        df_features = pd.concat([df_features, pd.DataFrame.from_records([dict_data])])
    return df_features



def get_features_per_year(data_map):
    for i in range(start_year, end_year):
        base_df = data_map[i]
        base_df['spotify_track_ids'] = generate_track_ids(base_df)
        df_features = generate_components_from_track_ids(base_df)
        base_df['danceability'] = list(df_features['danceability'])
        base_df['energy'] = list(df_features['energy'])
        base_df['speechiness'] = list(df_features['speechiness'])
        base_df['acousticness'] = list(df_features['acousticness'])
        base_df['liveness'] = list(df_features['liveness'])
        data_map[i] = base_df

get_features_per_year(data_map=data_map)

In [ ]:
from lyricsgenius import Genius

lg_client_id = 'To3XwrGWUtmcFjMLobriy0PAXL6RNAPaajsdNIXuorx2znvSAQLoio5LRBFg1u6P'
lg_client_secret = 'o2ncATH2ApRtYYuRUCphIOO5ijlQulQ9viFQ43JpUlopNddZLJJ7AB1HR8rSmK4mOPHk7JifAbyG5DAlbWKGOw'
lg_access_token = 'cHgP_TsmzzSHjbQ6Z-FVe2MEaxK7R4RFQ5L6jm4K8WumZ0ejysSuBTNdBwA3qvjF'

genius = Genius(access_token=lg_access_token, excluded_terms=["(Remix)", "(Live)"], remove_section_headers=True, verbose=False)


def gen_lyics_for_songs(data_map):
    for i in range(start_year, end_year):
        song_lyrics_lst= list()
        df = data_map[i]
        for ind in df.index:
            artist = df['Artist'][ind]
            track_name = df['Song Title'][ind]
            # Try and remove and search by each artist sequentially
            artist_list = generate_artists(artist_list=artist)
            found_song = False
            for curr_artist in artist_list:
                song = genius.search_song(track_name, artist=curr_artist)
                if(song):
                    song_lyrics_lst.append(song.lyrics)
                    found_song = True
                if(found_song == True):
                    break
            if(found_song == True):
                continue            
            # Search song without artist, take the topmost
            song = genius.search_song(track_name)
            if(not song):
                song_lyrics_lst.append("")
                continue
            song_lyrics_lst.append(song.lyrics)
        df['lyrics'] = song_lyrics_lst
        data_map[i] = df

gen_lyics_for_songs(data_map=data_map)

In [ ]:
%pip install ibm_watson
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('TKRJQ7w5AmtHUSLN0iCdwTF2fuXPzvYu9dY5Ebm3xzyh')
tone_analyzer = ToneAnalyzerV3(
    version='2017-09-21',
    authenticator=authenticator
)

tone_analyzer.set_service_url('https://api.au-syd.tone-analyzer.watson.cloud.ibm.com')
tone_analyzer.set_disable_ssl_verification(True)

# Set songs for tone